**Giải quyết Sentiment Analysis dùng Naive Bayes**

## Tải dữ liệu

Tải dữ liệu rồi cho dữ liệu vào trong pandas.dataframe

In [28]:
import pandas as pd
from os import listdir
from os.path import dirname
def load_data(dir):
    cmt={}
    cmt['sentiment'] = 0 if (dir=='D:/dataset/sav/data_train/train/practice_neg') else 1
    cmt['comment'] = []
    for filename in listdir(dir):
        file = open("{0}/{1}".format(dir, filename), 'r', encoding='utf-8')
        text = file.read().lower()
        cmt['comment'].append(text)
        file.close()
    return cmt
cmt_pos = load_data("D:/dataset/sav/data_train/train/practice_pos")
cmt_neg = load_data("D:/dataset/sav/data_train/train/practice_neg")

frame = pd.DataFrame(cmt_pos, columns=['sentiment', 'comment'])
frame_2 = pd.DataFrame(cmt_neg, columns=['sentiment', 'comment'])
frame = frame.append(frame_2, ignore_index = True)
frame

,sentiment,comment
0,1,xe_đẩy bán cơm_chiên nằm ngay đầu đường vào kh...
1,1,món ăn lạ mà ngon . quán mới ra 3 món tomocorn...
2,1,ghé quán_ăn mà đông quá_trời . tuy phục_vụ hơi...
3,1,"gía cả bình_dân , chất_lượng tốt . món ăn khá ..."
4,1,mình đã đến lacquer ăn trưa hò_hẹn với tụi bạn...
...,...,...
367,0,"chờ vừa lâu mà đồ_ăn vừa dở , chẳng có gì gọi_..."
368,0,k thích cách mấy nhân_viên phục_vụ - thiếu chu...
369,0,"sushi bình_thường , giá_cả lại không hợp_lý .\..."
370,0,"1 năm ăn lại , chắc tại tiệm đông khách nên kĩ..."


## Phân chia dữ liệu

Chia dữ liệu thành 2 tập train (X, y) và test (X_test, y_test)

Tập train (X, y) sau này sẽ dùng cho cross-validation k-fold

In [29]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(frame, test_size=0.1)
X = df_train['comment'].values
y = df_train['sentiment'].values
X_test = df_test['comment'].values
y_test = df_test['sentiment'].values
print(X_train.shape)
print(y_train.shape)

(300, 3506)
(300,)


In [30]:
print(X[0:5])
print(y[0:5])

['cảm_giác khj bước vào quán dc cả nhân_viên đến quản_lí đều hô chào bằng câu tiếng hàn rất thich thú\ncảm_giác như mình là trung_điểm của quán hjjj\nđồ_ăn ngon , vị lạ nhất_là món " gà sôt vị cay " : cay ơi_là cay , ngon ơi_là ngon\ncòn được tặng thêm những chú gà bông xinh và siêu kute luôn nữa chứ hjjj\ncác bạn thử 1 lần đến quán nha\nxem có như lời mình nói kô nhé ! !\nđảm_bảo đi 1 lần lại muốn đi lần nữa như mình ý 😘 😘 😘 😘 😘\n'
 'không_gian đẹp , khá ấm_cúng . giá hợp_lý nếu sử_dụng các gói student hay_là gói 40h . giá lẻ hơi đắt 😅 😅\n'
 'nay thèm sushi quá tình_cờ thấy quán này nên gọi ngay . vì có một_mình mình ăn nên chỉ gọi một phần mà quán cũng giao luôn . ship rẻ lắm . mà tận gần 22h . nhiệt_tình lắm . mình mua sushi chiên . vừa chiên xù nóng_hổi giòn_rụm nắp hộp còn đọng hơi_nước luôn . ngon lắm . sốt béo cay_cay ăn giờ còn ghiền . chắc mai sẽ kêu ăn tiếp . giá 29k mà 8 khoanh luôn , ăn no căng mà không ngán . mấy phần khác cũng 19-29-39k thôi à .\nđiện_thoại chụp hình khôn

## Làm sạch dữ liệu

Loại bỏ các ký tự đặc biệt & chữ số

In [47]:
import re
def cleandata(cmts):
    data = []
    for i in cmts:
        data.append(re.sub(r"[-()\"#/@;:<>{}=~|.?,\n0-9]", "", i))
    return data
X = cleandata(X)
X_test = cleandata(X_test)
X[0:5]

['cảm_giác khj bước vào quán dc cả nhân_viên đến quản_lí đều hô chào bằng câu tiếng hàn rất thich thúcảm_giác như mình là trung_điểm của quán hjjjđồ_ăn ngon  vị lạ nhất_là món  gà sôt vị cay   cay ơi_là cay  ngon ơi_là ngoncòn được tặng thêm những chú gà bông xinh và siêu kute luôn nữa chứ hjjjcác bạn thử  lần đến quán nhaxem có như lời mình nói kô nhé ! !đảm_bảo đi  lần lại muốn đi lần nữa như mình ý 😘 😘 😘 😘 😘',
 'không_gian đẹp  khá ấm_cúng  giá hợp_lý nếu sử_dụng các gói student hay_là gói h  giá lẻ hơi đắt 😅 😅',
 'nay thèm sushi quá tình_cờ thấy quán này nên gọi ngay  vì có một_mình mình ăn nên chỉ gọi một phần mà quán cũng giao luôn  ship rẻ lắm  mà tận gần h  nhiệt_tình lắm  mình mua sushi chiên  vừa chiên xù nóng_hổi giòn_rụm nắp hộp còn đọng hơi_nước luôn  ngon lắm  sốt béo cay_cay ăn giờ còn ghiền  chắc mai sẽ kêu ăn tiếp  giá k mà  khoanh luôn  ăn no căng mà không ngán  mấy phần khác cũng k thôi à điện_thoại chụp hình không đẹp lắm  chứ quán decor đẹp_mắt lắm có bán đem đi_ở 

In [32]:
print(len(y))

334


## Tạo vector bag-of-word cho mỗi câu bình luận

Lưu các bag-of-word vector dưới dạng dataframe

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(X)
bow_test = vectorizer.transform(X_test)
print(bow.toarray().shape)
vocab = vectorizer.get_feature_names()
print(len(vocab))

(334, 3586)
3586


In [35]:
df_bow = pd.DataFrame(bow.toarray(), columns = vocab)
df_bow.shape

(334, 3586)

In [36]:
df_bow.keys()

Index(['a_bảo', 'aaaaaaa', 'accoustics', 'acoustic', 'acousticlivemusic',
       'after', 'again', 'ah', 'ai', 'ai_ngờ',
       ...
       'ốc_bươu', 'ống', 'ốp', 'ồn', 'ồn_ào', 'ổn', 'ổng', 'ổnđiểm', 'ớt',
       'ủng_hộ'],
      dtype='object', length=3586)

In [60]:
df_bow_test = pd.DataFrame(bow_test.toarray(), columns = vocab)
df_bow_test.head()

,a_bảo,aaaaaaa,accoustics,acoustic,acousticlivemusic,after,again,ah,ai,ai_ngờ,...,ốc_bươu,ống,ốp,ồn,ồn_ào,ổn,ổng,ổnđiểm,ớt,ủng_hộ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Xây dựng & đánh giá chất lượng mô hình Naive Bayes

Thử nghiệm 2 phương pháp : Multinomial Naive Bayes vs Bernoulli Naive Bayes

Và dùng k-fold cross validation để chọn phương pháp nào hiệu quả hơn

In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
nb = MultinomialNB()
bnb = BernoulliNB()

In [49]:
# df = df_bow_train.append(df_bow_test)
y = np.array(y)
y

array([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,

In [50]:
from sklearn.model_selection import KFold
kfold = KFold(10, True, 1)
m_scores = []
b_scores = []
for train_index, test_index in kfold.split(df_bow_train):
    X_train, X_val, y_train, y_val = df_bow.iloc[train_index], df_bow.iloc[test_index], y[train_index], y[test_index]
    nb.fit(X_train, y_train)
    bnb.fit(X_train, y_train)
    m_scores.append(nb.score(X_val, y_val))
    b_scores.append(bnb.score(X_val, y_val))
print(np.mean(m_scores))
print(np.mean(b_scores))

0.8415329768270944
0.7994652406417113


In [51]:
print(m_scores)
print(b_scores)

[0.8823529411764706, 0.8529411764705882, 0.7941176470588235, 0.7647058823529411, 0.7878787878787878, 0.9090909090909091, 0.7272727272727273, 0.9393939393939394, 0.9393939393939394, 0.8181818181818182]
[0.8823529411764706, 0.6764705882352942, 0.7352941176470589, 0.8823529411764706, 0.7272727272727273, 0.7878787878787878, 0.696969696969697, 0.8484848484848485, 0.9090909090909091, 0.8484848484848485]


Ta có thể thấy Multinomial Naive Bayes cho chất lượng tốt hơn

## Dự đoán

In [52]:
y_mpredict = nb.predict(df_bow_test)
y_bpredict = bnb.predict(df_bow_test)
for i in range(20):
    print('Predicting class of cmt{}: multinomial {}; bernoulli {}'.format(i, str(y_mpredict[i]), str(y_bpredict[i]))) 
    print('Class of cmt{}:{}'.format(i,  str(y_test[i])))

Predicting class of cmt0: multinomial 1; bernoulli 1
Class of cmt0:1
Predicting class of cmt1: multinomial 1; bernoulli 1
Class of cmt1:0
Predicting class of cmt2: multinomial 0; bernoulli 0
Class of cmt2:0
Predicting class of cmt3: multinomial 1; bernoulli 0
Class of cmt3:1
Predicting class of cmt4: multinomial 1; bernoulli 0
Class of cmt4:1
Predicting class of cmt5: multinomial 1; bernoulli 0
Class of cmt5:1
Predicting class of cmt6: multinomial 1; bernoulli 1
Class of cmt6:1
Predicting class of cmt7: multinomial 0; bernoulli 0
Class of cmt7:0
Predicting class of cmt8: multinomial 0; bernoulli 0
Class of cmt8:1
Predicting class of cmt9: multinomial 1; bernoulli 0
Class of cmt9:1
Predicting class of cmt10: multinomial 0; bernoulli 0
Class of cmt10:0
Predicting class of cmt11: multinomial 0; bernoulli 0
Class of cmt11:0
Predicting class of cmt12: multinomial 1; bernoulli 1
Class of cmt12:1
Predicting class of cmt13: multinomial 1; bernoulli 1
Class of cmt13:1
Predicting class of cmt14: